## 🧪 MBTI 크롤링 데이터 제목과 무비렌즈 데이터 제목 맞추기

#### 다운 받은 데이터프레임 불러오기

In [511]:
import pandas as pd
import numpy as np

In [527]:
ml_title = pd.read_csv("ml_title.csv")
mbti_all_title = pd.read_csv("mbti_all_title.csv")
mbti_none_title = pd.read_csv("mbti_none_title.csv")

In [528]:
ml_title.head()

,title
0,toy story
1,jumanji
2,grumpier old men
3,waiting to exhale
4,father of the bride part ii


In [529]:
ml_title.isna().sum()

title    7
dtype: int64

In [530]:
ml_title = ml_title.dropna(axis=0)

In [531]:
mbti_all_title.head()

,Character,Contents,Votes,MBTI,Enneagram,category,img_src
0,Moana Waialiki,moana,2.7k/477,ENFJ,7w6,movie,https://static1.personality-database.com/profi...
1,Joy,inside out,2.5k/358,ENFJ,7w6,movie,https://static1.personality-database.com/profi...
2,Judy Hopps,zootopia,2.3k/350,ENFJ,1w2,movie,https://static1.personality-database.com/profi...
3,Elle Woods,legally blonde,1.9k/227,ENFJ,3w2,movie,https://static1.personality-database.com/profi...
4,Neil Perry,dead poets society,1.7k/222,ENFJ,4w3,movie,https://static1.personality-database.com/profi...


In [532]:
mbti_none_title = mbti_none_title.iloc[301:601]

In [533]:
mbti_none_title.head()

,Contents
301,the lord of the rings
302,wall-e
303,alice in wonderland / through the looking glass
304,500 days of summer
305,the before trilogy


In [548]:
mbti_none_title[mbti_none_title.Contents.map(change_dict).notna()]

,Contents
301,the lord of the rings
302,wall-e
303,alice in wonderland / through the looking glass
305,the before trilogy
306,the hunchback of notre dame
...,...
586,ocean's eleven trilogy
587,the motorcycle diaries
592,the age of innocence
596,the clique


In [553]:
len(change_dict)

163

In [551]:
change_dict.pop('Wall E')

'wall·e'

In [546]:
change_dict['wall-e']='wall·e'

In [549]:
len(change_dict)

164

#### 제목으로 유사도 측정 후 top 3 구해주는 함수

* title_list에는 무비렌즈 제목 리스트를 넣어주시고,
* title에는 MBTI 크롤링 데이터 제목을 하나씩 넣어주시면 됩니다.

-> 그럼 title_list에서 title과 제목이 유사한 제목 top 3가 결과로 나옵니다.

In [54]:
def content_based_filtering_jaccard(title_list, title):
    topn=20
    target_split_set = set(title.split())
    sim_list = []
    sim_df = pd.DataFrame(title_list, columns=['title'])
    for idx, movie in enumerate(title_list):
        title_split_set = set(movie.split())
        title_intersection = target_split_set.intersection(title_split_set)
        jac_sim = float(len(title_intersection)) / (len(target_split_set) + len(title_split_set) - len(title_intersection))
        sim_list.append(jac_sim)
    sim_df['jaccard_similarity'] = sim_list

    return sim_df.sort_values('jaccard_similarity', ascending=False).reset_index(drop=True)[:topn]

In [537]:
len(title_list)

62416

In [534]:
ml_title_list = list(ml_title['title'])
mbti_none_title_list = list(mbti_none_title['Contents'])

사용 예시

In [44]:
search_title='alice in wonderland'
ml_title[ml_title.title=dddd=search_title]

,title
1009,alice in wonderland
14393,alice in wonderland
15298,alice in wonderland
25576,alice in wonderland
32959,alice in wonderland
41181,alice in wonderland
49323,alice in wonderland


In [136]:
len([i for i in mbti_none_title_list if i.startswith('a ')])

17

In [123]:
len([i for i in mbti_none_title_list if not i.startswith('the')])

189

In [520]:
i="a dsifj asdfaj i the asdfli"

In [521]:
i[2:]

'dsifj asdfaj i the asdfli'

In [515]:
def process_the_list(mbti_none_title_list, title_list, change_dict):
    after_mbti_none_title_list = [i for i in mbti_none_title_list if not i.startswith('the')]
    the_none_list = [i for i in mbti_none_title_list if i.startswith('the ')]
    the_movie_list = [i[4:]+', the' for i in mbti_none_title_list if i.startswith('the')]
    print(f"기존 none title list {len(mbti_none_title_list)}개 중,")
    print(f"the 로 시작하는 영화 개수: {len(the_none_list)}")
    print(f"the 로 시작 안하는 영화 개수: {len(after_mbti_none_title_list)}")
    cnt=0
    for movie, the_movie in zip(the_none_list,the_movie_list):
        if the_movie in title_list:
            change_dict[movie] = the_movie
            cnt+=1
        else:
            after_mbti_none_title_list.append(movie)
    print(f"the로 시작하는 영화 the 지우고 뒤에 ,the 붙인게 movie lens에 있는 영화가 {cnt}개")
    print(f"프로세싱후 추가 처리해야줘야 할 mbti none title list는 {len(after_mbti_none_title_list)}개")
    return after_mbti_none_title_list, change_dict

In [522]:
def process_a_list(mbti_none_title_list, title_list, change_dict):
    after_mbti_none_title_list = [i for i in mbti_none_title_list if not i.startswith('a ')]
    a_none_list = [i for i in mbti_none_title_list if i.startswith('a ')]
    a_movie_list = [i[2:]+', a' for i in mbti_none_title_list if i.startswith('a ')]
    print(f"기존 none title list {len(mbti_none_title_list)}개 중,")
    print(f"a 로 시작하는 영화 개수: {len(a_none_list)}")
    print(f"a 로 시작 안하는 영화 개수: {len(after_mbti_none_title_list)}")
    cnt=0
    for movie, a_movie in zip(a_none_list,a_movie_list):
        if a_movie in title_list:
            change_dict[movie] = a_movie
            cnt+=1
        else:
            after_mbti_none_title_list.append(movie)
    print(f"a로 시작하는 영화 a 지우고 뒤에 ,a 붙인게 movie lens에 있는 영화가 {cnt}개")
    print(f"프로세싱후 추가 처리해야줘야 할 mbti none title list는 {len(after_mbti_none_title_list)}개")
    return after_mbti_none_title_list, change_dict

In [ ]:
change_dict=dict()

In [535]:
mbti_none_title_list, change_dict = process_the_list(mbti_none_title_list, title_list, change_dict)

기존 none title list 300개 중,
the 로 시작하는 영화 개수: 111
the 로 시작 안하는 영화 개수: 189
the로 시작하는 영화 the 지우고 뒤에 ,the 붙인게 movie lens에 있는 영화가 87개
프로세싱후 추가 처리해야줘야 할 mbti none title list는 213개


In [536]:
mbti_none_title_list, change_dict = process_a_list(mbti_none_title_list, title_list, change_dict)

기존 none title list 213개 중,
a 로 시작하는 영화 개수: 17
a 로 시작 안하는 영화 개수: 196
a로 시작하는 영화 a 지우고 뒤에 ,a 붙인게 movie lens에 있는 영화가 13개
프로세싱후 추가 처리해야줘야 할 mbti none title list는 200개


In [185]:
from imdb import IMDb
from tqdm import tqdm
ia = IMDb()

## 2020년 이후 영화 제거
def remove_after_2020_movie(mbti_none_title_list):
    after_mbti_none_title_list=[]
    after_2020_movie=[]
    for movie_name in tqdm(mbti_none_title_list):
        search_movie = ia.search_movie(movie_name)
        if search_movie:
            search_movie = search_movie[0] # 첫번째 검색 결과 가져오기
            movie = ia.get_movie(search_movie.getID()) # ID로 검색해서 영화 세부정보 가져오기
            search_movie_lower = search_movie['title'].lower()
            if movie_name == search_movie_lower and movie['year']>=2020:
                after_2020_movie.append(movie_name)
#                 print(search_movie,movie['year']) # 검색 결과 영화 출력
            else:
                after_mbti_none_title_list.append(movie_name)
    print(f"2020년 이후 개봉 영화 {len(after_2020_movie)}개")
    print(f"프로세싱후 추가 처리해야줘야 할 mbti none title list는 {len(after_mbti_none_title_list)}개")
    return after_mbti_none_title_list, after_2020_movie    

In [198]:
after_mbti_none_title_list, after_2020_movie = remove_after_2020_movie(mbti_none_title_list)

  1%|▎                                                             | 1/185 [00:07<21:55,  7.15s/it]

Freaky 2020


  2%|█                                                             | 3/185 [00:16<16:31,  5.45s/it]

Firebird 2021


  3%|█▋                                                            | 5/185 [00:28<16:24,  5.47s/it]

...

The Woman in the Window 2020


 99%|███████████████████████████████████████████████████████████▎| 183/185 [16:35<00:11,  5.50s/it]

The World to Come 2020


100%|████████████████████████████████████████████████████████████| 185/185 [16:45<00:00,  5.44s/it]

The Broken Hearts Gallery 2020
2020년 이후 개봉 영화 71개
프로세싱후 추가 처리해야줘야 할 mbti none title list는 112개


In [492]:
c1 =  ml_title.title.str.contains("legend")
c2 = ml_title.title.str.contains("hei")
ml_title[c1 & c2].title.tolist()

[]

In [493]:
title_list = ml_title_list
title = after_mbti_none_title_list[111] # 'harry potter / wizarding world films'
print(title)
result = content_based_filtering_jaccard(title_list, title)
result

IndexError: list index out of range

In [490]:
num=2
change_dict[title]=result.loc[num,'title']
print([(k,v) for k,v in change_dict.items()][-2:])
len(change_dict)

[('the fast and the furious', 'fast and the furious, the'), ('the fox and the hound', 'fox and the hound, the')]


162

In [482]:
search_title= 'charlie countryman'

change_dict[title]=search_title
print([(k,v) for k,v in change_dict.items()][-2:])
len(change_dict)

[('the pebble and the penguin', 'pebble and the penguin, the'), ('the necessary death of charlie countryman', 'charlie countryman')]


159

In [503]:
change_dict

{'the lord of the rings': 'lord of the rings, the',
 'Wall E': 'wall·e',
 'alice in wonderland / through the looking glass': 'alice through the looking glass',
 'the before trilogy': ['before sunrise', 'before sunset', 'before midnight'],
 'the hunchback of notre dame': 'hunchback of notre dame, the',
 'everything everywhere all at once': 'all at once',
 'major grom: the plague doctor': 'major grom',
 'the hangover trilogy': ['hangover part ii, the', 'hangover part iii, the'],
 'interview with the vampire': 'interview with the vampire: the vampire chronicles',
 'the princess bride': 'princess bride, the',
 'the virgin suicides': 'virgin suicides, the',
 'barbie mariposa': 'barbie mariposa and her butterfly fairy friends',
 'buffalo ‘66': "buffalo '66",
 'the silence of the lambs': 'silence of the lambs, the',
 'the lovely bones': 'lovely bones, the',
 'the secret life of walter mitty': 'secret life of walter mitty, the',
 'the truman show': 'truman show, the',
 "la vie d'adèle": 'blue 

In [460]:
change_dict[title]=['journey to the center of the earth', 'journey 2: the mysterious island']
print([(k,v) for k,v in change_dict.items()][-2:])
len(change_dict)

[('dan life', 'dan in real life'), ('journey to the center of the earth / journey 2: the mysterious island', ['journey to the center of the earth', 'journey 2: the mysterious island'])]


153

In [26]:
change_dict=dict()

movie_lens
- 중복 영화가 있을 경우 그냥 제목으로 넣음
ex) alice through the looking glass 중복 2개

- 시리즈인 경우 시리즈를 리스트로 넣음

In [495]:
len(change_dict)

162

In [496]:
import pickle

In [554]:
with open("mbti_movie_to_movilens_301_600.pkl","wb") as f:
    pickle.dump(change_dict,f)

In [555]:
with open("mbti_movie_to_movilens_301_600.pkl","rb") as f:
    aa = pickle.load(f)

In [556]:
aa==change_dict

True

In [ ]:
real_none_title = [
    '500 days of summer',
    'all too well', # 10분 단편 영화
    'mary and max',
    'drowning love',
    'goncharov', # interent meme movie?
    'barbie as the nutcracker',
    'mune: guardian of the moon',
    'mean girls 2', # mean girls는 있는데 2가 없네
    "la leggenda del pianista sull'oceano",
    "la boum",
    "pelíšky",
    "crazy little thing called love",
    "hababam sınıfı",
    "é fada!", # it's fairy 영제 없음
    "zathura: a space adventure",
    "hot wheels world race / acceleracers",
    "cry me a sad river",
    "g.f.darwin",
    "spookley the square pumpkin",
    "never look away",
    "a brighter summer day",
    "ed.so family", # 이런 영화 없음
    "k.g.f",
    "trędowata",
    "cats", # 2019
    "chicuarotes", # 2019
    "he won't kill, she won't die", # 2019
    "bartok the magnificent",
    "a coffee in berlin",
    "color out of space", # 2019
    "daniel isn't real", # 2019
    "detachment",
    "la notte",
    "perras 2011",
    "guns akimbo",
    "step up : highwater", # 이건 드라만데
    "peep ‘tv’ show",
    "kuch kuch hota hai",
    "four sisters and a wedding",
    "kabhi khushi kabhie gham...",
    "in space wth markiplier",
    "the pirates! in an adventure with scientists!",
    "the legend of hei", #2019    
                  ]
                   

In [199]:
after_2020_movie

['freaky',
 'firebird',
 'slumberland',
 'a boy called christmas',
 'eurovision song contest: the story of fire saga',
 'bones and all',
 'halloween ends',
 'nightbooks',
 'i fell in love like a flower bouquet',
 'horse girl',
 'black bear',
 'clouds',
 'megan',
 'through my window',
 'fire island',
 "ma rainey's black bottom",
 'aftersun',
 'ghostbusters: afterlife',
 'uncle frank',
 'men',
 'judas and the black messiah',
 'annette',
 'hatching',
 'love & gelato',
 'mrs. harris goes to paris',
 "don't tell a soul",
 'i came by',
 'antlers',
 'malcolm & marie',
 'stargirl',
 'birds of paradise',
 'bullet train',
 'promising young woman',
 'love and leashes',
 'in the heights',
 'happiest season',
 'strange world',
 'young adult matters',
 'lightyear',
 'yes day',
 'see how they run',
 'dolittle',
 'pink skies ahead',
 'godmothered',
 'godzilla vs. kong',
 'jingle jangle: a christmas journey',
 'marcel the shell with shoes on',
 'a perfect pairing',
 'an american pickle',
 'my best frie

In [504]:
after2020=['house of gucci','last night in soho',"i'm thinking of ending things",'tenet','blonde',
        'ammonite','summer of 85','your name engraved herein','cruella','my first summer',
        'confessions of an invisible girl','guillermo del toro’s pinocchio','confissões de uma garota excluída',
        'mavka. the forest song','mencuri raden saleh','depois do universo','spider-man - lotus',
        "nahuel y el libro mágico","o menino que matou meus pais / a menina que matou os pais",
        "eduardo e mônica","monster high : the movie", "upside down magic",
           
        
          ]

In [505]:
after_2020_movie.extend(after2020)

In [506]:
len(set(after_2020_movie))

93

위처럼 MBTI 크롤링 데이터 제목을 하나씩 무비렌즈 제목 데이터랑 비교해주신 다음에, 

1. 같은 영화이거나 시리즈 영화이면 제목을 동일하게 처리해주시고

2. 아예 관련이 없는 제목 top 3가 나온다면 'MBTI 크롤링 데이터 제목'을 따로 리스트(real_none_title)에 담아서 저장해주세요!

    -> 얘네는 따로 처리 필요!